In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [14:34<00:00, 16.76it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
11815,Swang,29,2017-02-02,Rae Sremmurd,https://open.spotify.com/track/6mapJIPnQ23RTAe...,United States,top200,SAME_POSITION,474991.000,NaN,"18 Jul 2014, 22:40",Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,Rae Sremmurd ( /ˈreɪ ʃrˈɪmɜːrd/) is an America...,Hip-Hop,rap,hip hop,trap,seen live,NaN,NaN,NaN,NaN,NaN,spotify:track:6mapJIPnQ23RTAevUoE0DL,0.681,0.314,8.000,-9.319,1.000,0.058,0.200,0.000,0.100,0.166,139.992,audio_features,6mapJIPnQ23RTAevUoE0DL,https://api.spotify.com/v1/tracks/6mapJIPnQ23R...,https://api.spotify.com/v1/audio-analysis/6map...,208120.000,4.000,21,354,hip-hop,hip-hop
261685,Die Young,171,2019-04-05,Roddy Ricch,https://open.spotify.com/track/3wLtYwtqvKK2ewe...,United States,top200,MOVE_DOWN,258961.000,"17 Aug 2021, 14:21","04 Jan 2019, 18:58","Rodrick Wayne Moore Jr., was born on October 2...","Rodrick Wayne Moore Jr., was born on October 2...",Hip-Hop,rap,trap,hip hop,american,Roddy Ricch,2010s,2018,NaN,NaN,spotify:track:3wLtYwtqvKK2ewelSVPeDK,0.688,0.656,4.000,-5.541,0.000,0.276,0.218,0.000,0.115,0.365,160.414,audio_features,3wLtYwtqvKK2ewelSVPeDK,https://api.spotify.com/v1/tracks/3wLtYwtqvKK2...,https://api.spotify.com/v1/audio-analysis/3wLt...,157657.000,4.000,22,87,hip-hop,hip-hop
392549,Angels Like You,132,2020-12-10,Miley Cyrus,https://open.spotify.com/track/1daDRI9ahBonbWD...,United States,top200,MOVE_UP,288057.000,"09 Dec 2020, 13:43","05 Apr 2006, 21:12",Miley Ray Cyrus (born Destiny Hope Cyrus) is a...,Miley Ray Cyrus (born Destiny Hope Cyrus) is a...,pop,female vocalists,Disney,dance,better than selena gomez,rock,soft rock,Ballad,pop rock,2020s,spotify:track:1daDRI9ahBonbWD8YcxOIB,0.672,0.642,5.000,-4.035,1.000,0.031,0.098,0.000,0.100,0.494,121.981,audio_features,1daDRI9ahBonbWD8YcxOIB,https://api.spotify.com/v1/tracks/1daDRI9ahBon...,https://api.spotify.com/v1/audio-analysis/1daD...,196453.000,4.000,33,24,pop,pop
232393,Nothing Breaks Like a Heart (feat. Miley Cyrus),3,2018-12-29,Mark Ronson,https://open.spotify.com/track/27rdGxbavYJeBph...,United States,viral50,MOVE_DOWN,NaN,"03 Dec 2018, 09:40","18 Jul 2006, 17:06",Mark Ronson (born 4 September 1975) is a soug...,Mark Ronson (born 4 September 1975) is a soug...,funk,pop,british,Hip-Hop,dj,pop,country,Miley Cyrus,Disco,dance,spotify:track:27rdGxbavYJeBphck5MZAF,0.601,0.794,7.000,-5.844,0.000,0.067,0.010,0.000,0.388,0.244,114.066,audio_features,27rdGxbavYJeBphck5MZAF,https://api.spotify.com/v1/tracks/27rdGxbavYJe...,https://api.spotify.com/v1/audio-analysis/27rd...,217467.000,4.000,1,123,funk,other
180307,Paranoid,16,2018-05-05,Post Malone,https://open.spotify.com/track/3fpVWegR6YOS1Yk...,United States,viral50,SAME_POSITION,NaN,"21 Oct 2020, 03:49","07 Jun 2015, 09:50","Austin Richard Post (born July 4, 1995), known...","Austin Richard Post (born July 4, 1995), known...",Hip-Hop,rap,trap,cloud rap,pop,Hip-Hop,rap,trap,hip hop,pop,spotify:track:3fpVWegR6YOS1Yk5HSMYIq,0.625,0.536,0.000,-5.428,1.000,0.078,0.038,0.000,0.167,0.253,159.914,audio_features,3fpVWegR6YOS1Yk5HSMYIq,https://api.spotify.com/v1/tracks/3fpVWegR6YOS...,https://api.spotify.com/v1/audio-analysis/3fpV...,221787.000,4.000,1,144,hip-hop,hip-hop


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.535
female       0.133
group        0.044
non-binary   0.030
Name: gender, dtype: float64

In [38]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
72840,Unforgettable,69,2017-09-28,Thomas Rhett,https://open.spotify.com/track/30MM5jWpUmOxtTN...,United States,top200,MOVE_DOWN,370872.000,NaN,"11 Mar 2012, 05:45","Thomas Rhett Akins, Jr. (born in Valdosta, Geo...","Thomas Rhett Akins, Jr. (born in Valdosta, Geo...",country,male vocalists,contemporary country,american,KKK Country,NaN,NaN,NaN,NaN,NaN,spotify:track:30MM5jWpUmOxtTNd9Ey5LZ,0.661,0.750,7.000,-5.497,1.000,0.031,0.030,0.000,0.085,0.828,130.077,audio_features,30MM5jWpUmOxtTNd9Ey5LZ,https://api.spotify.com/v1/tracks/30MM5jWpUmOx...,https://api.spotify.com/v1/audio-analysis/30MM...,157080.000,4.000,35,225,country,country,male,NaN
115225,Dance Monkey,20,2019-11-07,Tones And I,https://open.spotify.com/track/1rgnBhdG2JDFTbY...,United States,viral50,MOVE_UP,NaN,NaN,"25 Aug 2019, 11:51","Toni Watson, known professionally as Tones and...","Toni Watson, known professionally as Tones and...",pop,australian,indie pop,female vocalists,alternative,NaN,NaN,NaN,NaN,NaN,spotify:track:2XU0oxnq2qxCpomAAuJY8K,0.824,0.588,6.000,-6.400,0.000,0.092,0.692,0.000,0.149,0.513,98.027,audio_features,2XU0oxnq2qxCpomAAuJY8K,https://api.spotify.com/v1/tracks/2XU0oxnq2qxC...,https://api.spotify.com/v1/audio-analysis/2XU0...,209438.000,4.000,1,717,pop,pop,male,NaN
90756,20 Min,142,2020-11-07,Lil Uzi Vert,https://open.spotify.com/track/0uxSUdBrJy9Un0E...,United States,top200,MOVE_DOWN,260034.000,"26 Mar 2021, 14:59","21 Apr 2016, 19:11","Symere Bysil Woods (born July 31, 1995), known...","Symere Bysil Woods (born July 31, 1995), known...",trap,Hip-Hop,rap,hip hop,emo rap,trap,Hip-Hop,rap,hip hop,experimental,spotify:track:0uxSUdBrJy9Un0EYoBowng,0.773,0.750,8.000,-4.009,0.000,0.117,0.109,0.000,0.174,0.783,123.426,audio_features,0uxSUdBrJy9Un0EYoBowng,https://api.spotify.com/v1/tracks/0uxSUdBrJy9U...,https://api.spotify.com/v1/audio-analysis/0uxS...,220587.000,4.000,53,665,trap,trap,NaN,"July 31, 1995"
452060,What A Shame,47,2021-02-09,Leyla Blue,https://open.spotify.com/track/7FGIEAVXfegxngH...,United States,viral50,NEW_ENTRY,NaN,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Leyla+Blue...",,sad,latin,rnb,Sophie,charli xcx,NaN,NaN,NaN,NaN,NaN,spotify:track:7FGIEAVXfegxngH9Jg6VwA,0.699,0.463,0.000,-7.828,1.000,0.475,0.315,0.000,0.080,0.565,143.990,audio_features,7FGIEAVXfegxngH9Jg6VwA,https://api.spotify.com/v1/tracks/7FGIEAVXfegx...,https://api.spotify.com/v1/audio-analysis/7FGI...,177200.000,4.000,43,12,sad,other,NaN,NaN
231205,Nuketown (feat. Juice WRLD),59,2018-12-10,Ski Mask The Slump God,https://open.spotify.com/track/74lnM5V6ecvoTPV...,United States,top200,MOVE_UP,474969.000,"08 Dec 2019, 22:25","20 Jun 2017, 01:57","Stokeley Clevon Goulbourne (born April 18, 199...","Stokeley Clevon Goulbourne (born April 18, 199...",Hip-Hop,trap,rap,florida,seen live,Juice WRLD,hip hop,trap,10s,bars,spotify:track:74lnM5V6ecvoTPV0fvptx9,0.808,0.617,10.000,-9.320,0.000,0.442,0.113,0.000,0.734,0.632,149.962,audio_features,74lnM5V6ecvoTPV0fvptx9,https://api.spotify.com/v1/tracks/74lnM5V6ecvo...,https://api.spotify.com/v1/audio-analysis/74ln...,166400.000,4.000,12,169,hip-hop,hip-hop,male,"April 18, 1996"


In [39]:
df[df['artist'] == 'Lil Uzi Vert'].to_csv('test.csv', sep = ';')

Age from artist

-- taking the data from the summary

In [31]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d{1,2}, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

December 2, 1998      10839
July 4, 1995          10700
January 23, 1998       9272
October 24, 1986       8719
December 18, 2001      7645
                      ...  
March 17, 1978            1
December 12, 1990         1
March 11, 1950            1
October 6, 2000           1
September 18, 1991        1
Name: birthday, Length: 988, dtype: int64

In [32]:
df.sample()

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
131860,when the party's over,82,2019-11-10,Billie Eilish,https://open.spotify.com/track/43zdsphuZLzwA9k...,United States,top200,MOVE_UP,288073.000,"19 Aug 2020, 07:25","15 Aug 2016, 17:51",Billie Eilish Pirate Baird O'Connell (born Dec...,Billie Eilish Pirate Baird O'Connell (born Dec...,pop,indie pop,female vocalists,electropop,indie,pop,Ballad,indie pop,2010s,alternative,spotify:track:43zdsphuZLzwA9k4DJhU0I,0.367,0.111,4.000,-14.084,1.000,0.097,0.978,0.000,0.090,0.198,82.642,audio_features,43zdsphuZLzwA9k4DJhU0I,https://api.spotify.com/v1/tracks/43zdsphuZLzw...,https://api.spotify.com/v1/audio-analysis/43zd...,196077.000,4.000,5,624,pop,pop,female,"December 18, 2001"


# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df

Genre distibution

How was the music during covid